# Imports

In [0]:
from pyspark.sql.functions import col, lit, acos, sin, cos, radians, broadcast

# Configuration

In [0]:
# Tables
AIRBNB_DATA_TABLE = "Itav_airbnb_hex_data"
BOOKING_DATA_TABLE = "Itav_booking_hex_data"
WORLD_CUP_DATA_TABLE = "world_cup_anastasia"
AIRBNB_DATA_OUTPUT_TABLE = "Itav_airbnb_world_cup_data"
BOOKING_DATA_OUTPUT_TABLE = "Itav_booking_world_cup_data"

# Parameters
DISTANCE_THRESHOLD_KM = 10.0
EARTH_RADIUS_KM = 6371.0

# Methodology

In [0]:
def filter_data_by_stadium_proximity(data_table_name, stadium_table_name):
    # Load data
    df_data = spark.table(data_table_name)
    df_stadiums = spark.table(stadium_table_name).select(
        col("city").alias("nearest_stadium_city"),
        col("stadium_name"),
        col("latitude").alias("stadium_lat"),
        col("longitude").alias("stadium_long")
    ).distinct()

    # Carsing product
    joined_df = df_data.crossJoin(broadcast(df_stadiums))

    # Distance filtering
    distance_expr = (
        acos(
            sin(radians(col("lat"))) * sin(radians(col("stadium_lat"))) +
            cos(radians(col("lat"))) * cos(radians(col("stadium_lat"))) *
            cos(radians(col("stadium_long")) - radians(col("long")))
        ) * lit(EARTH_RADIUS_KM)
    )
    distance_df = joined_df.withColumn("distance_to_stadium", distance_expr)
    filtered_df = distance_df.filter(col("distance_to_stadium") <= DISTANCE_THRESHOLD_KM)

    return filtered_df

# Create tables
itav_airbnb_world_cup_data = filter_data_by_stadium_proximity(AIRBNB_DATA_TABLE, WORLD_CUP_DATA_TABLE)
itav_booking_world_cup_data = filter_data_by_stadium_proximity(BOOKING_DATA_TABLE, WORLD_CUP_DATA_TABLE)

# Save tables
spark.conf.set("spark.databricks.delta.columnMapping.mode", "name")
itav_airbnb_world_cup_data.write.mode("overwrite").saveAsTable(AIRBNB_DATA_OUTPUT_TABLE)
itav_booking_world_cup_data.write.mode("overwrite").option("delta.columnMapping.mode", "name").saveAsTable(BOOKING_DATA_OUTPUT_TABLE)

# Validation
print("Airbnb filtered data:")
display(itav_airbnb_world_cup_data.orderBy("distance_to_stadium").limit(10))

Airbnb filtered data:


h3_index name image description category availability discount reviews seller_info breadcrumbs location pets_allowed description_items category_rating house_rules details highlights arrangement_details amenities images available_dates url final_url listing_title property_id listing_name location_details description_by_sections description_html location_details_html is_supperhost host_rating hosts_year host_response_rate is_guest_favorite travel_details pricing_details currency cancellation_policy country postcode_map_url host_image host_details price total_price host_number_of_reviews property_number_of_reviews guests ratings lat long flag_no_reviews flag_suspicious_text flag_bad_price safety_alert nightlife_score nature_score relaxation_score shopping_score tourist_score nearest_stadium_city stadium_name stadium_lat stadium_long distance_to_stadium 8a44c1a8cc27fff Rental unit in Atlanta · 2 bedrooms · 2 beds · 1 bath https://a0.muscache.com/pictures/35c449d9-85e2-42f7-bde5-3adad9742792.jpg Main living spaceKitchenDinning areaMash RoomQueen BenKing Bed1 BathOther things to noteLOCATED 1.2 MILES OF THE MERCEDES BENZ STADIUM! Stays false null null {"seller_id":"157089993","url":"https://www.airbnb.com/users/show/157089993"} Atlanta, Georgia, United States Atlanta, Georgia, United States false ["Entire rental unit","2 beds","1 bath"] [] ["Check-in: 11:00 a.m.–1:00 p.m.","Checkout before 11:00 a.m.","4 guests maximum"] ["4 guests","2 bedrooms","2 beds","1 bath"] [] null [{"group_name":"Bedroom and laundry","items":[{"name":"Washer","value":"SYSTEM_WASHER"},{"name":"Dryer","value":"SYSTEM_DRYER"},{"name":"Essentials","value":"Towels, bed sheets, soap, and toilet paper"},{"name":"Iron","value":"SYSTEM_IRON"}]},{"group_name":"Entertainment","items":[{"name":"TV with standard cable","value":"SYSTEM_TV"}]},{"group_name":"Heating and cooling","items":[{"name":"Air conditioning","value":"SYSTEM_SNOWFLAKE"},{"name":"Heating","value":"SYSTEM_THERMOMETER"}]},{"group_name":"Home safety","items":[{"name":"Smoke alarm","value":"SYSTEM_DETECTOR_SMOKE"},{"name":"Fire extinguisher","value":"SYSTEM_FIRE_EXTINGUISHER"}]},{"group_name":"Internet and office","items":[{"name":"Wifi","value":"SYSTEM_WI_FI"}]},{"group_name":"Kitchen and dining","items":[{"name":"Kitchen","value":"Space where guests can cook their own meals"}]},{"group_name":"Location features","items":[{"name":"Private entrance","value":"Separate street or building entrance"}]},{"group_name":"Parking and facilities","items":[{"name":"Free parking on premises","value":"SYSTEM_MAPS_CAR_RENTAL"},{"name":"Pool","value":"SYSTEM_POOL"},{"name":"Gym","value":"SYSTEM_GYM"}]},{"group_name":"Not included","items":[]}] ["https://a0.muscache.com/im/pictures/35c449d9-85e2-42f7-bde5-3adad9742792.jpg"] [] https://www.airbnb.ca/rooms/21597505?currency=USD&guests=1 https://www.airbnb.ca/rooms/21597505?currency=USD&guests=1&locale=en&enable_auto_translate=true&source_impression_id=p3_1747590661_P3tabNy7Dc2BaLYd Condo Campbell 21597505 Entire rental unit in Atlanta, Georgia, United States [{"title":"Getting around","value":"This community is located within walking distance of TWO public transportation train stations. Georgia State and MLK MARTA Stations."}] [{"title":null,"value":"Main living spaceKitchenDinning areaMash RoomQueen BenKing Bed1 Bath"},{"title":"Other things to note","value":"LOCATED 1.2 MILES OF THE MERCEDES BENZ STADIUM!"}] 
 
Main living space
 
Kitchen
 
Dinning area
 
Mash Room
 
Queen Ben
 
King Bed
 
1 Bath
 
 
 Other things to note 
 
LOCATED 1.2 MILES OF THE MERCEDES BENZ STADIUM!
 
 undefined false 0 null null false {"check_in":null,"check_out":null,"num_of_adults":null,"num_of_children":null,"num_of_infants":null,"num_of_pets":null} {"airbnb_service_fee":null,"cleaning_fee":null,"initial_price_per_night":null,"num_of_nights":null,"price_per_night":null,"price_without_fees":null,"special_offer":null,"taxes":null} USD null null null null null 0.0 0.0 0.0 0.0 4.0 0.0 33.75611 -84.40